In [1]:
#Import the Dependencies form the tensorflow dataset
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [2]:
# Initialize Hyperparameters
IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 50

In [3]:
#Import data into tensorflow dataset object form the folders saved in local PC

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "d:\Documents\My_Projects\DeepLearning\potatoDiseaseClassification_Project3",
    shuffle=True, 
    image_size=(IMAGE_SIZE, IMAGE_SIZE),  # 256 * 256 size
    batch_size= BATCH_SIZE
)
#The datasets are in three different folders/classes : healthy leaves, diseased leaves with Early Blight, Late Blight

Found 2152 files belonging to 3 classes.


In [4]:
# Names of classes are read in digit format. Finding their class names
class_names = dataset.class_names
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [5]:
for image_batch, labels_batch in dataset.take(1):
    print(image_batch.shape)
    print(labels_batch.numpy())

#Each element in the dataset is a tuple. First element is a batch of 32 elements of images. Second element is a batch of 32 elements of class labels

(32, 256, 256, 3)
[1 0 0 1 1 1 0 1 0 0 1 1 0 0 0 1 0 1 2 1 1 0 0 1 1 1 1 0 1 0 0 0]


Running the visulisation cell will shut down the kernel, So prevent doing so.
This maybe because of loading the high constrast, high defination images that cell is not able to handel

In [6]:
# Visualize some of the images from the dataset

# plt.figure(figsize=(10, 10))
# for image_batch, labels_batch in dataset.take(1):
#     for i in range(12):
#         ax = plt.subplot(3, 4, i + 1)
#         plt.imshow(image_batch[i].numpy().astype("uint8"))
#         plt.title(class_names[labels_batch[i]])
#         plt.axis("off")

#Now, we write the function to split the dataset into 3 types

Training: Dataset to be used while training --> 80%
Validation: Dataset to be tested against while training --> 10%
Test: Dataset to be tested against after we trained a model --> 10%

In [7]:
len(dataset) #We have 32 batches and each batch has 68 images : 38 * 68 (to accomodate all images in the batches)

68

In [8]:
train_size = 0.8
len(dataset)*train_size

54.400000000000006

In [9]:
train_ds = dataset.take(54)
len(train_ds)

54

In [10]:
test_ds = dataset.skip(54)
len(test_ds)

14

In [11]:
val_size=0.1
len(dataset)*val_size

6.800000000000001

In [12]:
val_ds = test_ds.take(6)
len(val_ds)

6

In [13]:
test_ds = test_ds.skip(6)
len(test_ds)

8

In [14]:
# Combining all the concepts from the above, we define the function

def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    ds_size = len(ds)
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [15]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

Cache, Shuffle, and Prefetch the Dataset

In [16]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [17]:
# Moreover, to improve model performance, we should normalize the image pixel value (keeping them in range 0 and 1 by dividing by 256).

resize_and_rescale = tf.keras.Sequential([
    #Resize the images if not in the dataset to 256 * 256 during prediction 
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE), #256 * 256
    # Scale the image to 255
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

Create the image variations from the single image ->  Change in constrast, position of image, flip the image, zoom.
This is done so that when we are training we train all variations of the single image we have better prediction.

Data Augmentation is needed when we have less data, this boosts the accuracy of our model by augmenting the data.

In [18]:

data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2)
])

Building the Model : Model Architecture

We use a CNN coupled with a Softmax activation in the output layer. We also add the initial layers for resizing, normalization and Data Augmentation.

We are going to use convolutional neural network (CNN) which is popular for image classification tasks. 

In [19]:

input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 3

model = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=input_shape)

In [20]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (32, 256, 256, 3)         0         
                                                                 
 conv2d (Conv2D)             (32, 254, 254, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (32, 127, 127, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (32, 125, 125, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (32, 62, 62, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (32, 60, 60, 64)         

Compiling the Model : 
We use adam Optimizer, SparseCategoricalCrossentropy for losses, accuracy as a metric

In [21]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [34]:
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=10,
)

Epoch 1/10


54/54 [==============================] - 96s 2s/step - loss: 0.3909 - accuracy: 0.8484 - val_loss: 0.6671 - val_accuracy: 0.7500
Epoch 2/10
54/54 [==============================] - 102s 2s/step - loss: 0.3372 - accuracy: 0.8628 - val_loss: 0.1668 - val_accuracy: 0.9271
Epoch 3/10
54/54 [==============================] - 93s 2s/step - loss: 0.2054 - accuracy: 0.9288 - val_loss: 0.0884 - val_accuracy: 0.9740
Epoch 4/10
54/54 [==============================] - 69s 1s/step - loss: 0.2007 - accuracy: 0.9288 - val_loss: 0.1150 - val_accuracy: 0.9635
Epoch 5/10
54/54 [==============================] - 79s 1s/step - loss: 0.1414 - accuracy: 0.9514 - val_loss: 0.0861 - val_accuracy: 0.9583
Epoch 6/10
54/54 [==============================] - 73s 1s/step - loss: 0.1110 - accuracy: 0.9595 - val_loss: 0.0900 - val_accuracy: 0.9896
Epoch 7/10
54/54 [==============================] - 70s 1s/step - loss: 0.0757 - accuracy: 0.9734 - val_loss: 0.0187 - val_accuracy: 1.0000
Epoch 8/10
54/54 [============

In [35]:
history

history.params

history.history.keys()




dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [36]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [25]:
# plt.figure(figsize=(8, 8))
# plt.subplot(1, 2, 1)
# plt.plot(range(EPOCHS), acc, label='Training Accuracy')
# plt.plot(range(EPOCHS), val_acc, label='Validation Accuracy')
# plt.legend(loc='lower right')
# plt.title('Training and Validation Accuracy')

# plt.subplot(1, 2, 2)
# plt.plot(range(EPOCHS), loss, label='Training Loss')
# plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
# plt.legend(loc='upper right')
# plt.title('Training and Validation Loss')
# plt.show()

We will now Run prediction on a sample image

In [39]:
import numpy as np

for images_batch, labels_batch in test_ds.take(1):
    first_image = image_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()

    print("First image to predict")
    first_image
    #The actual label of the image
    print("Actual Label: ", class_names[first_label])

    #To predict the batch which contains 32 images
    batch_prediction = model.predict(images_batch)
    #To predict the batch prediction of the first image from that batch
    print("predicted label: ", class_names[np.argmax(batch_prediction[0])])
    




First image to predict
Actual Label:  Potato___Early_blight
1/1 [==============================] - 0s 420ms/step
predicted label:  Potato___Early_blight


Below function is for the inference. We will try to run the inference on few sample images in the next cell.

In [40]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [41]:
# plt.figure(flgsize=(15,15))
for images, labels in test_ds.take(1):
    for i in range(9): # Here we take only 9 images to test
        # ax = plt.subplot(3, 3, i+1)
        predicted_class, confidence = predict(model, images[i].numpy)
        #Actual class of that specific image
        actual_class = class_names[labels[i]]

        



1/1 [==============================] - 0s 55ms/step


Save The Model 
We will add the model to the list of models as a new version every time

In [44]:
# Save the model in the folder models 
# One can save the multiple version of the model also

model_version = 1
model.save(f"./models/{model_version}")


INFO:tensorflow:Assets written to: ./models/1\assets


INFO:tensorflow:Assets written to: ./models/1\assets


In [56]:
# If we want to have multiple models and want to increment the model version automatically
# We just import the os library and auto increment the model number in the models folder.

import os
model_version=max([int(i) for i in os.listdir("./saved_model_versions/") + [0]])+1
model.save(f"./saved_model_versions/odels/{model_version}")

INFO:tensorflow:Assets written to: ./saved_model_versions/odels/3\assets


INFO:tensorflow:Assets written to: ./saved_model_versions/odels/3\assets
